# Fan In / Fan Out — Report

This notebook visualizes the CSV outputs generated by the **Fan_In_Fan_Out** block.

## What this notebook shows
- **Top Fan-In** classes (Bar).
- **Top Fan-Out** classes (Bar).
- **Fan-In vs Fan-Out** scatter (each dot = class).
- **Distributions** for both metrics (Histograms).
- **Ratio** view (Fan-In / (Fan-Out + 1)) to spot potential hotspots (Bar).

> If a CSV is missing or empty, the cell prints an info message and skips the chart.


In [ ]:
# Setup: imports, paths, helpers
# - CSVs are read from reports/csv-reports/<CATEGORY>/<file>.csv relative to this notebook folder.
# - Minimal console output; only show information if a CSV is missing/empty.
# - Bar charts use an explicit default color so it's easy to tweak later.
# - Titles are standardized without block prefixes.

import os
from pathlib import Path
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go

pd.set_option('future.no_silent_downcasting', True)

CATEGORY = "Fan_In_Fan_Out"
CSV_BASE = Path("../reports/csv-reports").resolve()
FIO_DIR = CSV_BASE / CATEGORY

# Explicit default color for all bar charts in this notebook
DEFAULT_BAR_COLOR = ["#1f77b4"]

# CSV IO helper
NA_LITS = ["", " ", "NA", "N/A", "n/a", "NaN", "NULL", "Null", "null", "None", "none", "-", "--"]

def read_csv_safe(p: Path) -> pd.DataFrame:
    """Read a CSV if present; otherwise return an empty DataFrame.
    Prints a minimal info message when missing or unreadable."""
    p = Path(p)
    if not p.exists():
        print(f"[info] Missing CSV: {p}")
        return pd.DataFrame()
    try:
        df = pd.read_csv(p, na_values=NA_LITS, keep_default_na=True)
        df.columns = [str(c).strip() for c in df.columns]
        df = df.dropna(how="all")
        return df
    except Exception as e:
        print(f"[warn] Failed to read {p}: {e}")
        return pd.DataFrame()

def find_col(df, *cands, default=None, contains=None):
    """Find a column by exact candidates or by substring (contains)."""
    low = {c.lower(): c for c in df.columns}
    for c in cands:
        if c and c.lower() in low:
            return low[c.lower()]
    if contains:
        for k, orig in low.items():
            if contains.lower() in k:
                return orig
    return default

# Import shared chart functions from interface
import sys
sys.path.append(str(Path("../interface").resolve()))
from charts.fan_in_fan_out_charts import (
    load_and_merge_fanin_fanout,
    create_top_fanin_bar,
    create_top_fanout_bar,
    create_fanin_vs_fanout_scatter,
    create_fanin_distribution,
    create_fanout_distribution,
    create_ratio_bar
)


## 1) Load & unify metrics

In [ ]:
# Load Fan_In.csv and Fan_Out.csv, then merge on class/type
# No charts here — just data preparation for the following sections.

# Read CSVs
path_in = FIO_DIR / "Fan_In.csv"
df_in = read_csv_safe(path_in)

path_out = FIO_DIR / "Fan_Out.csv"
df_out = read_csv_safe(path_out)

# Use shared function to merge data
merged = load_and_merge_fanin_fanout(df_in, df_out)

display(merged.head(10))


## 2) Top Fan-In classes

In [ ]:
# Chart generated here:
#  - 2A) Top classes by Fan-In (Bar, explicit color)

if merged.empty:
    print("[info] No data available for Fan-In / Fan-Out.")
else:
    fig = create_top_fanin_bar(merged)
    if fig:
        fig.show()
    else:
        print("[info] No data for top Fan-In bar")


## 3) Top Fan-Out classes

In [ ]:
# Chart generated here:
#  - 3A) Top classes by Fan-Out (Bar, explicit color)

if not merged.empty:
    fig = create_top_fanout_bar(merged)
    if fig:
        fig.show()
    else:
        print("[info] No data for top Fan-Out bar")


## 4) Fan-In vs Fan-Out — scatter

In [ ]:
# Chart generated here:
#  - 4A) Fan-In vs Fan-Out (Scatter, size = fanIn + fanOut)

if not merged.empty:
    fig = create_fanin_vs_fanout_scatter(merged)
    if fig:
        fig.show()
    else:
        print("[info] No data for scatter plot")


## 5) Distributions — histograms

In [ ]:
# Charts generated here:
#  - 5A) Distribution of Fan-In (Histogram)
#  - 5B) Distribution of Fan-Out (Histogram)

if not merged.empty:
    fig1 = create_fanin_distribution(merged)
    if fig1:
        fig1.show()
    else:
        print("[info] No data for Fan-In distribution")

    fig2 = create_fanout_distribution(merged)
    if fig2:
        fig2.show()
    else:
        print("[info] No data for Fan-Out distribution")


## 6) Ratio view — Fan-In / (Fan-Out + 1)

In [ ]:
# Chart generated here:
#  - 6A) Top classes by Fan-In to Fan-Out ratio (Bar, explicit color)

if not merged.empty:
    fig = create_ratio_bar(merged)
    if fig:
        fig.show()
    else:
        print("[info] No data for ratio bar chart")
